In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
#!pip install --upgrade numpy

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
            os.path.join(dirname, filename)
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import Rescaling, RandomZoom, RandomFlip, RandomRotation
from sklearn.model_selection import train_test_split
import os
from distutils.dir_util import copy_tree, remove_tree
!pip install imblearn
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

  Using cached imblearn-0.0-py2.py3-none-any.whl (1.9 kB)
  Using cached imbalanced_learn-0.10.1-py3-none-any.whl (226 kB)


In [2]:
base_dir = "/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/"
root_dir = "./"
test_dir = base_dir + "test/"
train_dir = base_dir + "train/"
work_dir = root_dir + "dataset/"

if os.path.exists(work_dir):
    remove_tree(work_dir)
    

os.mkdir(work_dir)
copy_tree(train_dir, work_dir)
copy_tree(test_dir, work_dir)
print("Working Directory Contents:", os.listdir(work_dir))

WORK_DIR = './dataset/'


DistutilsFileError: cannot copy tree '/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/': not a directory

In [3]:

os.listdir(WORK_DIR)



['NonDemented', 'MildDemented', 'VeryMildDemented', 'ModerateDemented']

In [4]:
import tensorflow as tf
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)
    
print(tf.__version__)

Device: 
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local
INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


Number of replicas: 8
2.12.0


In [5]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import Rescaling, RandomZoom, RandomFlip, RandomRotation
from sklearn.model_selection import train_test_split

IMG_SIZE = 176
IMAGE_SIZE = [176, 176]
DIM = (IMG_SIZE, IMG_SIZE)



# Create the data augmentation pipeline
data_augmentation = tf.keras.Sequential([
    Rescaling(1./255),
    RandomZoom(0.2),
    RandomFlip("horizontal"),
    RandomRotation(0.2)
])

# Load images and apply data augmentation
train_data_gen = tf.keras.preprocessing.image_dataset_from_directory(
    directory=WORK_DIR,
    image_size=DIM,
    batch_size=64,
    shuffle=False,
    seed=42,
    validation_split=None,
    subset=None,
labels="inferred"
)

train_data_augmented = train_data_gen.map(lambda x, y: (data_augmentation(x, training=True), y))




Found 6400 files belonging to 4 classes.


In [6]:
train_data = []
train_labels = []

for data, labels in train_data_augmented:
    train_data.append(data)
    train_labels.append(labels)

train_data = tf.concat(train_data, axis=0)
train_labels = tf.concat(train_labels, axis=0)


In [7]:
num_samples = train_data.shape[0]
train_data_2d = tf.reshape(train_data, (num_samples, -1))


In [8]:
# Perform over-sampling of the data using SMOTE

# Apply SMOTE-IMG
smote_img = SMOTE(sampling_strategy='minority')



In [9]:


train_data_resampled, train_labels_resampled = smote_img.fit_resample(train_data_2d, train_labels)

# Reshape train_data_resampled back to 4D
train_data_resampled = train_data_resampled.reshape(-1, IMG_SIZE, IMG_SIZE, 3)

In [10]:
train_data_resampled.shape

(9536, 176, 176, 3)

In [11]:



# Split the augmented data into train and test sets
train_data1, test_data1, train_labels1, test_labels1 = train_test_split(train_data_resampled, train_labels_resampled, test_size=0.2, random_state=42,stratify=train_labels_resampled)

# Further split the train set into train and validation sets
train_data1, val_data1, train_labels1, val_labels1 = train_test_split(train_data1, train_labels1, test_size=0.2, random_state=42,stratify=train_labels1)



In [12]:
from keras.utils import to_categorical

train_labels1 = to_categorical(train_labels1)
val_labels1 = to_categorical(val_labels1)

import matplotlib.pyplot as plt
Name_of_the_classes = train_labels1

plt.figure(figsize=(10, 10))
for image_batch, labels_batch in train_data1.take(1):
    print(image_batch.shape)
    print(labels_batch.numpy())
    for i in range(12):
        ax = plt.subplot(4, 3, i+1)
        plt.axis('off')
        plt.title(Name_of_the_classes[labels_batch[i]])
        plt.imshow(image_batch[i].numpy().astype('uint8'))
plt.show()


In [13]:
def conv_block(filters):
    block = tf.keras.Sequential([
        tf.keras.layers.SeparableConv2D(filters, 3, activation='relu', padding='same'),
        tf.keras.layers.SeparableConv2D(filters, 3, activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPool2D()
    ]
    )
    
    return block

def dense_block(units, dropout_rate):
    block = tf.keras.Sequential([
        tf.keras.layers.Dense(units, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(dropout_rate)
    ])
    
    return block

In [14]:
def build_model():
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(IMAGE_SIZE[0],IMAGE_SIZE[1], 3)),
        
        tf.keras.layers.Conv2D(16, 3, activation='relu', padding='same'),
        tf.keras.layers.Conv2D(16, 3, activation='relu', padding='same'),
        tf.keras.layers.MaxPool2D(),
        
        conv_block(32),
        conv_block(64),
        
        conv_block(128),
        tf.keras.layers.Dropout(0.2),
        
        conv_block(256),
        tf.keras.layers.Dropout(0.2),
        
        tf.keras.layers.Flatten(),
        dense_block(512, 0.7),
        dense_block(128, 0.5),
        dense_block(64, 0.3),
        
        tf.keras.layers.Dense(4,activation='softmax')
    ])
    
    return model

In [15]:
with strategy.scope():
    model = build_model()

    METRICS = [tf.keras.metrics.AUC(name='auc')]
    
    model.compile(
        optimizer='adam',
        loss=tf.losses.CategoricalCrossentropy()
        ,
        metrics=METRICS
    )
    
    
    model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 176, 176, 16)      448       
                                                                 
 conv2d_1 (Conv2D)           (None, 176, 176, 16)      2320      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 88, 88, 16)       0         
 )                                                               
                                                                 
 sequential_1 (Sequential)   (None, 44, 44, 32)        2160      
                                                                 
 sequential_2 (Sequential)   (None, 22, 22, 64)        7392      
                                                                 
 sequential_3 (Sequential)   (None, 11, 11, 128)       27072     
                                                      

In [16]:
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 **(epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(0.01, 20)

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("alzheimer_model.h5",
                                                    save_best_only=True)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10,
                                                     restore_best_weights=True)

In [17]:
history = model.fit(train_data1,train_labels1,
    validation_data=(val_data1, val_labels1),
    callbacks=[checkpoint_cb, early_stopping_cb, lr_scheduler],
    epochs=100
)


Epoch 1/100


2023-06-30 11:23:37.903834: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add_100/ReadVariableOp.
2023-06-30 11:23:38.151197: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add_100/ReadVariableOp.


191/191 [==============================] - ETA: 0s - loss: 1.4016 - auc: 0.6191

2023-06-30 11:24:05.327104: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.
2023-06-30 11:24:05.454530: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.


191/191 [==============================] - 44s 125ms/step - loss: 1.4016 - auc: 0.6191 - val_loss: 1.3795 - val_auc: 0.6471 - lr: 0.0100
Epoch 2/100
191/191 [==============================] - 6s 29ms/step - loss: 1.2883 - auc: 0.6682 - val_loss: 1.4164 - val_auc: 0.6072 - lr: 0.0089
Epoch 3/100
191/191 [==============================] - 5s 29ms/step - loss: 1.2636 - auc: 0.6859 - val_loss: 28.3331 - val_auc: 0.5577 - lr: 0.0079
Epoch 4/100
191/191 [==============================] - 6s 32ms/step - loss: 1.2575 - auc: 0.6900 - val_loss: 1.2757 - val_auc: 0.6770 - lr: 0.0071
Epoch 5/100
191/191 [==============================] - 5s 29ms/step - loss: 1.2341 - auc: 0.7079 - val_loss: 2.3195 - val_auc: 0.6164 - lr: 0.0063
Epoch 6/100
191/191 [==============================] - 6s 29ms/step - loss: 1.1987 - auc: 0.7315 - val_loss: 6.6321 - val_auc: 0.5622 - lr: 0.0056
Epoch 7/100
191/191 [==============================] - 6s 29ms/step - loss: 1.1929 - auc: 0.7344 - val_loss: 3.4454 - val_auc: 

AttributeError: module 'keras.api._v2.keras.models' has no attribute 'load'

In [7]:
import tensorflow as tf
from keras.models import load_model

In [9]:
model = load_model("alzheimer_model.h5" , compile=False)

In [ ]:
model.predict